In [2]:
!pip install bitfinex-tencars

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
import time

import bitfinex

In [20]:
# Create api instance of the v2 API
api_v2 = bitfinex.bitfinex_v2.api_v2()

No keys, only access to public API functions


In [21]:
# test connection, get open, high, low, close for 1 minute BTC data --> last 1000 minutes
result = api_v2.candles()

In [22]:
# Define query parameters
pair = 'btcusd'  # currency pair of interest, Bitcoin USD pair
bin_size = '1m'  # 1m = minutes data
limit = 1000     # maximum of 1000 data points

# define start date
t_start = datetime.datetime(2018, 4, 1, 0, 0)
t_start = time.mktime(t_start.timetuple()) * 1000

# define end date
t_stop = datetime.datetime(2018, 4, 2, 0, 0)
t_stop = time.mktime(t_stop.timetuple()) * 1000

result = api_v2.candles(
    symbol = pair,
    interval = bin_size,
    limit = limit,
    start = t_start,
    end = t_stop
)

In [23]:
# create a function to get more than 1000 data points
def fetch_data(start, stop, symbol, interval, tick_limit, step):
    """
    Lets us run larger queries, need to provide step size in milliseconds
    """
    api_v2 = bitfinex.bitfinex_v2.api_v2()
    data = []
    start = start - step
    while start < stop:
        start = start + step
        end = start + step
        res = api_v2.candles(
            symbol = symbol,
            interval = interval,
            limit = tick_limit,
            start = start,
            end = end
        )
        data.extend(res)
        time.sleep(2)
    return data

In [32]:
# set step size
time_step = 60000000

# define start date
t_start = datetime.datetime(2020, 1, 1, 0, 0)
t_start = time.mktime(t_start.timetuple()) * 1000

# define end date
t_stop = datetime.datetime(2020, 3, 24, 0, 0)
t_stop = time.mktime(t_stop.timetuple()) * 1000

pair_data = fetch_data(
    start = t_start,
    stop = t_stop,
    symbol = pair,
    interval = bin_size,
    tick_limit = limit,
    step = time_step
)

No keys, only access to public API functions


In [43]:
pair_data = pair_data[:-4]

In [45]:
# create a pandas data frame from results of api call
cols = [
    'time',
    'open',
    'close',
    'high',
    'low',
    'volume'
]

df = pd.DataFrame(pair_data, columns = cols)
df.drop_duplicates(inplace=True)
df['time'] = pd.to_datetime(df['time'], unit='ms')
df.set_index('time', inplace=True)
df.sort_index(inplace=True)
df.shape

(119306, 5)

In [46]:
df.head()

,open,close,high,low,volume
time,,,,,
2020-01-01 08:00:00,7219.700000,7219.643269,7219.700000,7219.643269,0.003531
2020-01-01 08:01:00,7219.643269,7219.643269,7219.643269,7219.643269,0.006944
2020-01-01 08:02:00,7219.643269,7219.600000,7219.643269,7219.600000,0.015000
2020-01-01 08:03:00,7219.697041,7219.697041,7219.697041,7219.697041,0.200000
2020-01-01 08:04:00,7219.697041,7219.697041,7219.697041,7219.697041,0.010001


In [47]:
df.tail()

,open,close,high,low,volume
time,,,,,
2020-03-25 08:36:00,6730.1,6723.0,6730.1,6718.779674,12.439059
2020-03-25 08:37:00,6723.0,6730.1,6730.1,6723.000000,7.295661
2020-03-25 08:38:00,6730.1,6722.9,6730.1,6722.900000,1.404563
2020-03-25 08:39:00,6722.9,6724.9,6724.9,6722.900000,1.823615
2020-03-25 08:40:00,6723.6,6720.0,6723.6,6720.000000,1.634946


In [48]:
df.to_csv('2020_btc_minutes_data')